# Neural Style Transfer

### Implementation of : https://arxiv.org/pdf/1508.06576.pdf

In [55]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
import pprint
%matplotlib inline

In [56]:
model = scipy.io.loadmat("pretrainedModel/imagenet-vgg-verydeep-19.mat")

In [64]:
# Function that computes the content cost
def compute_content_cost(a_C, a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_C_unrolled = tf.reshape(a_C,shape=[m,n_H*n_W,n_C])
    a_G_unrolled = tf.reshape(a_G,shape=[m,n_H*n_W,n_C])
    
    j_content = (1/(4 * n_H*n_W*n_C))*tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled,a_G_unrolled)))
    return j_content

In [93]:
def generate_noise_image(content_image, noise_ratio = 0.6):
    """
    Generates a noisy image by adding random noise to the content_image
    """

    noise_image = np.random.uniform(-20, 20, (1, 400, 300, 3)).astype('float32')
    
    # Set the input_image to be a weighted average of the content_image and a noise_image
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    
    return input_image

In [94]:
def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """
    GA = tf.matmul(A,tf.transpose(A))
    
    
    return GA

In [96]:
def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S = tf.reshape(tf.transpose(a_S), shape=[n_C,n_H*n_W])
    a_G = tf.reshape(tf.transpose(a_G), shape=[n_C,n_H*n_W])

    
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

 
    J_style_layer = (1/(4*(n_C**2) * (n_H*n_W)**2))*tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(GS,GG))))
    
    return J_style_layer

<tf.Tensor: id=1461, shape=(), dtype=float32, numpy=5.7224073>